Equations of motion for a Top, "z" is the axis about which the body rotates. 
Euler angles (ZXZ) are used. 


```
  R (m, J_G)
   \     
    \  r  (along z_b)
     \  
      O
```

In [ ]:
# IPython and standard packages
import numpy as np
import matplotlib.pyplot as plt
from sympy.physics.vector import init_vprinting
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))
InteractiveShell.ast_node_interactivity = "all"
get_ipython().run_line_magic('matplotlib', 'inline')
get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')
init_vprinting(use_latex='mathjax', pretty_print=False)
%matplotlib inline
# Sympy
import sympy as sp
from sympy.physics.mechanics import dynamicsymbols
# Yams sympy
from welib.yams.yams_sympy       import YAMSRigidBody, YAMSInertialBody, YAMSFlexibleBody
from welib.yams.yams_sympy_model import YAMSModel


In [ ]:
# --- Main symbols
theta, phi, psi = dynamicsymbols('theta, phi, psi')
omegax, omegay, omegaz = dynamicsymbols('omega_x, omega_y, omega_z')
Mx, My, Mz = dynamicsymbols('M_x, M_y, M_z') # Torques on Top
theta0, phi0, Ktheta, Kphi = sp.symbols('theta_0, phi_0, K_theta, K_phi') # Restoring forces
r = sp.symbols('r')
m = sp.symbols('m')
gravity = sp.symbols('g') # Gravity
time = dynamicsymbols._t
Jxx, Jyy, Jzz, JO, JG = sp.symbols('Jxx, Jyy, Jzz, J_O, J_G')

In [ ]:
# --- Options
BodyStartAtO=False
InertiaAtO=True
# --- Bodies and connections
ref = YAMSInertialBody('E') 
if BodyStartAtO:
    # --- OPTION 1: the body is at the origin, but we specify that the COG is at "R"
    if InertiaAtO:
        body = YAMSRigidBody('B', mass=m, J_G=(JO,JO,Jzz), rho_G=(0,0,r), J_at_Origin=True) # Create a body named "B"
    else:        
        body = YAMSRigidBody('B', mass=m, J_G=(JG,JG,Jzz), rho_G=(0,0,r), J_at_Origin=False) # Create a body named "B"
    ref.connectTo(body, type='Free', rel_pos=(0, 0, 0), rot_type='Body', rot_amounts=(phi,  theta, psi), rot_order='ZXZ')
else:
    # --- OPTION 2: the body is at "r" away from the origin
    if InertiaAtO:
        body = YAMSRigidBody('B', mass=m, J_G=(JO-m*r**2,JO-m*r**2,Jzz), rho_G=(0,0,0)) # Create a body named "B"
    else:
        body = YAMSRigidBody('B', mass=m, J_G=(JG,JG,Jzz), rho_G=(0,0,0)) # Create a body named "B"
    ref.connectTo(body, type='Free', rel_pos_b=(0, 0, r), rot_type='Body', rot_amounts=(phi,  theta, psi), rot_order='ZXZ')
bodies=[body] 
# --- Frames (to define loads)
gyro_frame_nonrot= ref.frame.orientnew('nr', 'Body', (phi,  theta, 0), 'ZXZ') # phi=yaw, theta=tilt
gyro_frame       = ref.frame.orientnew('r',  'Body', (phi,  theta, psi), 'ZXZ')
# --- DOFs and kinetic equations
coordinates = [phi, theta, psi]
speeds      = [omegax, omegay, omegaz]
kdeqsSubs = body.kdeqsSubsOmega((omegax,omegay,omegaz), ref_frame=ref.frame, out_frame=body.frame)
kdeqs = [om_symb-om_value for om_symb,om_value in kdeqsSubs]
print('coordinates:');coordinates
print('kdeqsSubs  :'); kdeqsSubs
# --- Reinserting kinetic equations 
body.update_ang_vel((omegax,omegay,omegaz), ref_frame=ref.frame, out_frame=body.frame)

In [ ]:
# --- DEBUG print
#body.frame.ang_vel_in(body.inertial_frame).express(body.inertial_frame).subs(kdeqsSubs).simplify()
#body.printPosVel(kdeqsSubs=kdeqsSubs)
#body.central_inertia.to_matrix(body.frame)

In [ ]:
# --- Loads
body_loads = []
for bdy in bodies: # gravity
    body_loads  += [(bdy, (bdy.masscenter, -bdy.mass * gravity * ref.frame.z))]
print('TODO FIGURE OUT IN WHICH FRAME MOMENTS ARE APPLIED')
body_loads += [(body, (body.frame, Mx * gyro_frame_nonrot.x + My * gyro_frame_nonrot.y + Mz * body.frame.z))] # ext torques
body_loads += [(body, (body.frame, -Ktheta*(theta-theta0) * gyro_frame_nonrot.x ))]
body_loads += [(body, (body.frame, -Kphi*(phi-phi0) * ref.frame.z))]
# --- Model and equations of motions
model = YAMSModel(ref=ref, bodies=bodies, body_loads=body_loads, 
                  coordinates=coordinates, speeds=speeds, kdeqsSubs=kdeqsSubs)
model.kaneEquations()
print('fr')
fr=model.kane.fr
fr.simplify()
fr
print('frstar')
model.kane.frstar
print('Kanes full coordiantes')
model.q_full
print('Kanes full mass matrix')
MM = model.kane.mass_matrix_full
MM.simplify()
MM
print('Kanes full forcing')
FF = model.kane.forcing_full
FF.simplify()
FF
#print('Equations of motions')
#EOM=model.to_EOM()
#EOM.EOM
#EOM.mass_forcing_form()
#print('Mass matrix:')
#EOM.M
#print('Forcing term:')
#EOM.F

In [ ]:
# --- Simulation
frames_per_sec = 20.
final_time = 15.0
time =  np.linspace(0.0, final_time, int(final_time * frames_per_sec))
u = {Mx:0, My:0, Mz:0}
p = {m:10., gravity:9.81, JO:100., Jzz:100., r:1.0, phi0:30.* np.pi/180, theta0:(90-5)*np.pi/180, Ktheta:0, Kphi:0}
q0 = {phi: p[phi0], theta:p[theta0], omegaz:1.12}
y = model.integrate(time, q0, p, u=u, M=MM, F=FF)
plt.plot(time, np.mod(y[:, :3]*180/np.pi,360));
plt.xlabel('Time [s]');
plt.ylabel('Angle [deg]');
plt.legend(["${}$".format(sp.latex(c)) for c in model.coordinates]);

In [ ]:
import os
os.getcwd() # sometime pydy creates a bunch of directories, keep an eye on it
#os.chdir(r'C:\W0\Work\2018-NREL\_libs\welib\welib\yams\examples_symbolic_kane')
body.viz_opts={'type':'cylinder','radius':10, 'length':0.01,'normal':'z', 'color':'white'}
#body.viz_opts={'type':'three-blades','radius':10, 'normal':'z', 'color':'white'}
scene = model.getScene(rp=0.5, al=1)
scene.display()
#display(scene.display_jupyter()) # jupyter lab
#scene.display_ipython(); HTML(scene._html_widget.value)
